In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("="*80)
print("COVID-19 CLINICAL TRIALS - EXPLORATORY DATA ANALYSIS")
print("="*80)


COVID-19 CLINICAL TRIALS - EXPLORATORY DATA ANALYSIS


In [9]:
# 1. LOAD DATASET

print("\n[Step 1] Loading Dataset...")
try:
    df = pd.read_csv('/content/drive/MyDrive/COVID clinical trials.csv', index_col=0)
    print(f"✓ Dataset loaded successfully!")
    print(f"  Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")

# 2. INITIAL DATA EXPLORATION

    print("\n[Step 2] Initial Data Exploration")
    print("-"*80)

    print("\n2.1 First 5 Rows:")
    print(df.head())

    print("\n2.2 Dataset Information:")
    print(df.info())

    print("\n2.3 Statistical Summary (Numerical):")
    print(df.describe())

    print("\n2.4 Statistical Summary (Categorical):")
    print(df.describe(include='object'))

    print("\n2.5 Column Names:")
    print(df.columns.tolist())

    # 3. DATA TYPES ANALYSIS

    print("\n[Step 3] Data Types Analysis")
    print("-"*80)

    categorical_cols = df.select_dtypes(include='object').columns.tolist()
    numerical_cols = df.select_dtypes(exclude='object').columns.tolist()

    print(f"\nCategorical Features ({len(categorical_cols)}):")
    print(categorical_cols)

    print(f"\nNumerical Features ({len(numerical_cols)}):")
    print(numerical_cols)

    # 4. MISSING DATA ANALYSIS

    print("\n[Step 4] Missing Data Analysis")
    print("-"*80)

    missing_data = df.isnull().sum()
    missing_percent = (df.isnull().mean() * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing_Count': missing_data,
        'Missing_Percentage': missing_percent
    }).sort_values('Missing_Percentage', ascending=False)

    print("\nMissing Data Summary:")
    print(missing_df[missing_df['Missing_Count'] > 0])

    # Visualize Missing Data
    plt.figure(figsize=(15, 8))
    missing_percent_sorted = missing_percent.sort_values(ascending=False)
    sns.barplot(x=missing_percent_sorted.values, y=missing_percent_sorted.index, palette='viridis')
    plt.xlabel('Percentage of Missing Data', fontsize=12)
    plt.ylabel('Features', fontsize=12)
    plt.title('Missing Data Analysis - Percentage by Feature', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('1_missing_data_analysis.png', dpi=300, bbox_inches='tight')
    print("\n✓ Saved: 1_missing_data_analysis.png")
    plt.close()

    # 5. DATA CLEANING

    print("\n[Step 5] Data Cleaning")
    print("-"*80)

    # Drop columns with >95% missing data
    high_missing = missing_percent[missing_percent > 95].index.tolist()
    if high_missing:
        print(f"\nDropping columns with >95% missing data: {high_missing}")
        df.drop(columns=high_missing, inplace=True)

    # Update categorical_cols after dropping columns
    categorical_cols = df.select_dtypes(include='object').columns.tolist()

    # Check for duplicates
    print(f"\nDuplicate rows before removal: {df.duplicated().sum()}")
    df.drop_duplicates(inplace=True)
    print(f"Duplicate rows after removal: {df.duplicated().sum()}")

    # Extract Country from Locations
    if 'Locations' in df.columns:
        df['Country'] = df['Locations'].apply(
            lambda x: str(x).split(',')[-1].strip() if pd.notna(x) else 'Unknown'
        )
        print("✓ Extracted 'Country' feature from 'Locations'")

    # Handle missing data in categorical features
    cat_features_with_missing = df[categorical_cols].columns[df[categorical_cols].isnull().any()].tolist()
    for col in cat_features_with_missing:
        if col in df.columns:
            df[col].fillna(f'Missing {col}', inplace=True)
            print(f"✓ Filled missing values in '{col}' with 'Missing {col}'")

    # Handle missing data in numerical features (Enrollment)
    if 'Enrollment' in df.columns and df['Enrollment'].isnull().any():
        median_enrollment = df['Enrollment'].median()
        df['Enrollment'].fillna(median_enrollment, inplace=True)
        print(f"✓ Filled missing 'Enrollment' values with median: {median_enrollment}")

    print(f"\n✓ Data Cleaning Complete!")
    print(f"  Final Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")

    # 6. UNIVARIATE ANALYSIS

    print("\n[Step 6] Univariate Analysis")
    print("-"*80)

    # 6.1 Status Distribution
    if 'Status' in df.columns:
        plt.figure(figsize=(12, 6))
        status_counts = df['Status'].value_counts()
        sns.barplot(x=status_counts.index, y=status_counts.values, palette='Set2')
        plt.xlabel('Status', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.title('Distribution of Clinical Trial Status', fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('2_status_distribution.png', dpi=300, bbox_inches='tight')
        print("\n✓ Saved: 2_status_distribution.png")
        plt.close()

        print("\nStatus Distribution:")
        print(status_counts)

    # 6.2 Phases Distribution
    if 'Phases' in df.columns:
        plt.figure(figsize=(12, 6))
        phases_counts = df['Phases'].value_counts().head(10)
        sns.barplot(x=phases_counts.index, y=phases_counts.values, palette='coolwarm')
        plt.xlabel('Phases', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.title('Distribution of Clinical Trial Phases (Top 10)', fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('3_phases_distribution.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 3_phases_distribution.png")
        plt.close()

    # 6.3 Age Group Distribution
    if 'Age' in df.columns:
        plt.figure(figsize=(12, 6))
        age_counts = df['Age'].value_counts()
        sns.barplot(x=age_counts.index, y=age_counts.values, palette='muted')
        plt.xlabel('Age Group', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.title('Distribution of Age Groups in Clinical Trials', fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('4_age_distribution.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 4_age_distribution.png")
        plt.close()

    # 6.4 Gender Distribution
    if 'Gender' in df.columns:
        plt.figure(figsize=(10, 6))
        gender_counts = df['Gender'].value_counts()
        colors = sns.color_palette('pastel')[0:len(gender_counts)]
        plt.pie(gender_counts.values, labels=gender_counts.index, autopct='%1.1f%%',
                colors=colors, startangle=90)
        plt.title('Gender Distribution in Clinical Trials', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('5_gender_distribution.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 5_gender_distribution.png")
        plt.close()

    # 6.5 Top 15 Countries
    if 'Country' in df.columns:
        plt.figure(figsize=(14, 8))
        top_countries = df['Country'].value_counts().head(15)
        sns.barplot(y=top_countries.index, x=top_countries.values, palette='viridis')
        plt.xlabel('Number of Trials', fontsize=12)
        plt.ylabel('Country', fontsize=12)
        plt.title('Top 15 Countries by Number of Clinical Trials', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('6_top_countries.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 6_top_countries.png")
        plt.close()

        print("\nTop 15 Countries:")
        print(top_countries)

    # 6.6 Study Type Distribution
    if 'Study Type' in df.columns:
        plt.figure(figsize=(10, 6))
        study_type_counts = df['Study Type'].value_counts()
        sns.barplot(x=study_type_counts.index, y=study_type_counts.values, palette='Set3')
        plt.xlabel('Study Type', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.title('Distribution of Study Types', fontsize=14, fontweight='bold')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('7_study_type_distribution.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 7_study_type_distribution.png")
        plt.close()

    # 6.7 Enrollment Distribution
    if 'Enrollment' in df.columns:
        plt.figure(figsize=(12, 6))

        # Remove outliers for better visualization
        q1 = df['Enrollment'].quantile(0.25)
        q3 = df['Enrollment'].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        enrollment_filtered = df[(df['Enrollment'] >= lower_bound) & (df['Enrollment'] <= upper_bound)]['Enrollment']

        plt.subplot(1, 2, 1)
        plt.hist(enrollment_filtered, bins=50, color='skyblue', edgecolor='black')
        plt.xlabel('Enrollment', fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        plt.title('Enrollment Distribution (Outliers Removed)', fontsize=12, fontweight='bold')

        plt.subplot(1, 2, 2)
        plt.boxplot(enrollment_filtered, vert=True)
        plt.ylabel('Enrollment', fontsize=12)
        plt.title('Enrollment Boxplot', fontsize=12, fontweight='bold')

        plt.tight_layout()
        plt.savefig('8_enrollment_distribution.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 8_enrollment_distribution.png")
        plt.close()

        print("\nEnrollment Statistics:")
        print(df['Enrollment'].describe())

    # 7. BIVARIATE ANALYSIS

    print("\n[Step 7] Bivariate Analysis")
    print("-"*80)

    # 7.1 Status vs Phases
    if 'Status' in df.columns and 'Phases' in df.columns:
        plt.figure(figsize=(14, 8))

        # Get top statuses and phases
        top_statuses = df['Status'].value_counts().head(5).index
        top_phases = df['Phases'].value_counts().head(5).index

        df_filtered = df[df['Status'].isin(top_statuses) & df['Phases'].isin(top_phases)]
        status_phase = pd.crosstab(df_filtered['Status'], df_filtered['Phases'])

        status_phase.plot(kind='bar', stacked=True, colormap='tab10', figsize=(12, 6))
        plt.xlabel('Status', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.title('Status vs Phases (Top 5 Each)', fontsize=14, fontweight='bold')
        plt.legend(title='Phases', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('9_status_vs_phases.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 9_status_vs_phases.png")
        plt.close()

    # 7.2 Status vs Study Type
    if 'Status' in df.columns and 'Study Type' in df.columns:
        plt.figure(figsize=(12, 6))
        status_study = pd.crosstab(df['Status'], df['Study Type'])
        status_study.plot(kind='bar', colormap='Paired', figsize=(12, 6))
        plt.xlabel('Status', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.title('Status vs Study Type', fontsize=14, fontweight='bold')
        plt.legend(title='Study Type', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('10_status_vs_study_type.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 10_status_vs_study_type.png")
        plt.close()

    # 7.3 Gender vs Age
    if 'Gender' in df.columns and 'Age' in df.columns:
        plt.figure(figsize=(12, 6))
        gender_age = pd.crosstab(df['Gender'], df['Age'])
        gender_age.plot(kind='bar', colormap='Set2', figsize=(12, 6))
        plt.xlabel('Gender', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.title('Gender vs Age Group', fontsize=14, fontweight='bold')
        plt.legend(title='Age', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('11_gender_vs_age.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 11_gender_vs_age.png")
        plt.close()

    # 8. TIME SERIES ANALYSIS

    print("\n[Step 8] Time Series Analysis")
    print("-"*80)

    if 'Start Date' in df.columns:
        # Convert to datetime
        df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')

        # Extract month and year
        df['Start_Month'] = df['Start Date'].dt.to_period('M')
        df['Start_Year'] = df['Start Date'].dt.year

        # Trials started over time (monthly)
        plt.figure(figsize=(16, 6))
        trials_over_time = df['Start_Month'].value_counts().sort_index()
        trials_over_time.plot(kind='line', marker='o', color='dodgerblue', linewidth=2)
        plt.xlabel('Month', fontsize=12)
        plt.ylabel('Number of Trials', fontsize=12)
        plt.title('Clinical Trials Started Over Time (Monthly)', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('12_trials_over_time_monthly.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 12_trials_over_time_monthly.png")
        plt.close()

        # Trials by year
        plt.figure(figsize=(12, 6))
        trials_by_year = df['Start_Year'].value_counts().sort_index()
        sns.barplot(x=trials_by_year.index, y=trials_by_year.values, palette='rocket')
        plt.xlabel('Year', fontsize=12)
        plt.ylabel('Number of Trials', fontsize=12)
        plt.title('Clinical Trials Started by Year', fontsize=14, fontweight='bold')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('13_trials_by_year.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 13_trials_by_year.png")
        plt.close()

        print("\nTrials by Year:")
        print(trials_by_year)

    # 9. ADVANCED INSIGHTS

    print("\n[Step 9] Advanced Insights")
    print("-"*80)

    # 9.1 Top Sponsors
    if 'Sponsor/Collaborators' in df.columns:
        sponsors = df['Sponsor/Collaborators'].str.split('|', expand=True)[0]
        top_sponsors = sponsors.value_counts().head(15)

        plt.figure(figsize=(14, 8))
        sns.barplot(y=top_sponsors.index, x=top_sponsors.values, palette='magma')
        plt.xlabel('Number of Trials', fontsize=12)
        plt.ylabel('Sponsor', fontsize=12)
        plt.title('Top 15 Sponsors/Organizations', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('14_top_sponsors.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 14_top_sponsors.png")
        plt.close()

        print("\nTop 15 Sponsors:")
        print(top_sponsors)

    # 9.2 Funding Sources
    if 'Funded Bys' in df.columns:
        plt.figure(figsize=(12, 6))
        funding = df['Funded Bys'].value_counts()
        colors = sns.color_palette('husl', len(funding))
        plt.pie(funding.values, labels=funding.index, autopct='%1.1f%%',
                colors=colors, startangle=90)
        plt.title('Distribution of Funding Sources', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('15_funding_sources.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 15_funding_sources.png")
        plt.close()

    # 9.3 Correlation Analysis (if multiple numerical columns exist)
    numerical_df = df.select_dtypes(include=[np.number])
    if len(numerical_df.columns) > 1:
        plt.figure(figsize=(10, 8))
        correlation_matrix = numerical_df.corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                    square=True, linewidths=1, fmt='.2f')
        plt.title('Correlation Matrix - Numerical Features', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('16_correlation_matrix.png', dpi=300, bbox_inches='tight')
        print("✓ Saved: 16_correlation_matrix.png")
        plt.close()

    # 10. SAVE CLEANED DATA

    print("\n[Step 10] Saving Cleaned Data")
    print("-"*80)

    output_filename = 'cleaned_covid_clinical_trials.csv'
    df.to_csv(output_filename)
    print(f"✓ Cleaned dataset saved as: {output_filename}")

    # 11. SUMMARY REPORT

    print("\n" + "="*80)
    print("SUMMARY REPORT")
    print("="*80)

    print(f"\n📊 Dataset Overview:")
    print(f"   • Total Records: {df.shape[0]:,}")
    print(f"   • Total Features: {df.shape[1]}")
    print(f"   • Categorical Features: {len(df.select_dtypes(include='object').columns)}")
    print(f"   • Numerical Features: {len(df.select_dtypes(exclude='object').columns)}")

    if 'Status' in df.columns:
        most_common_status = df['Status'].mode()[0]
        print(f"\n📈 Trial Status:")
        print(f"   • Most Common Status: {most_common_status}")
        print(f"   • Unique Statuses: {df['Status'].nunique()}")

    if 'Country' in df.columns:
        top_country = df['Country'].value_counts().index[0]
        print(f"\n🌍 Geographic Distribution:")
        print(f"   • Top Country: {top_country} ({df['Country'].value_counts().iloc[0]} trials)")
        print(f"   • Total Countries: {df['Country'].nunique()}")

    if 'Enrollment' in df.columns:
        total_enrollment = df['Enrollment'].sum()
        avg_enrollment = df['Enrollment'].mean()
        print(f"\n👥 Enrollment Statistics:")
        print(f"   • Total Enrollment: {total_enrollment:,.0f}")
        print(f"   • Average Enrollment per Trial: {avg_enrollment:,.0f}")

    if 'Phases' in df.columns:
        print(f"\n🔬 Trial Phases:")
        print(f"   • Unique Phases: {df['Phases'].nunique()}")

    print(f"\n📁 Generated Files:")
    print(f"   • Analysis plots: 16 visualization files")
    print(f"   • Cleaned dataset: {output_filename}")

    print("\n" + "="*80)
    print("✅ EXPLORATORY DATA ANALYSIS COMPLETED SUCCESSFULLY!")
    print("="*80)
    print("\n💡 Next Steps:")
    print("   1. Review all generated PNG files for insights")
    print("   2. Use the cleaned dataset for machine learning models")
    print("   3. Consider deeper analysis on specific features of interest")
    print("   4. Document key findings for presentation\n")

except FileNotFoundError:
    print("ERROR: Please ensure 'COVID clinical trials.csv' is in the correct directory")
    exit()


[Step 1] Loading Dataset...
✓ Dataset loaded successfully!
  Dataset Shape: 5783 rows × 26 columns

[Step 2] Initial Data Exploration
--------------------------------------------------------------------------------

2.1 First 5 Rows:
       NCT Number                                              Title  \
Rank                                                                   
1     NCT04785898  Diagnostic Performance of the ID Now™ COVID-19...   
2     NCT04595136  Study to Evaluate the Efficacy of COVID19-0001...   
3     NCT04395482  Lung CT Scan Analysis of SARS-CoV2 Induced Lun...   
4     NCT04416061  The Role of a Private Hospital in Hong Kong Am...   
5     NCT04395924         Maternal-foetal Transmission of SARS-Cov-2   

           Acronym                  Status         Study Results  \
Rank                                                               
1      COVID-IDNow  Active, not recruiting  No Results Available   
2         COVID-19      Not yet recruiting  No Results A

<Figure size 1400x800 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>